In [ ]:
import numpy as np
from numpy import linalg as la
def VanillaOption_Explicit(
                  S_0,
                  T,
                  K,
                  r,
                  q,
                  sigma,
                  N,
                  M,
                  isCall,
                  isAmerican):
    outVector = np.zeros((2,1),dtype = float)
    S_max = 2.0*K
    S_min = 0.0
    #Discretization of the Stock and the Time
    dS=(S_max-S_min)/float(M)   
    dt=T/float(N)
    
    j=np.arange(0,M,dtype=np.float)   
    
    #'Transition Probabilities'
    a=(-0.5*(r-q)*dt*j+0.5*sigma**2*dt*j**2)/(1.0+r*dt)
    b=(1.0-sigma**2*dt*j**2)/(1.0+r*dt)
    c=(0.5*(r-q)*dt*j+0.5*sigma**2*dt*j**2)/(1.0+r*dt)
    #The matrix from the transition probabilities
    A=np.diag(b)+np.diag(a[1:],k=-1)+np.diag(c[0:M-1],k=1)
    infinityNormOfA = la.norm(A,np.inf)
    outVector[1] = infinityNormOfA
    #The matrix for the option price
    #N+1: Time Steps
    #M+1: Stock Steps
    FV=np.zeros((N+1,M+1)) 
    #Set up the boundary conditions//Contract dependent :-(
    if(bool(isCall)):
        FV[:,0]=S_min
        FV[:,M]=[S_max * np.exp(-r*( N - j)*dt) for j in range(N+1)]
        FV[N,:]=np.maximum(np.arange(S_min,S_max+dS/2.0,dS,dtype=np.float)-K,0)
    else:
        FV[:,0]=[K * np.exp(-r*( N - j)*dt) for j in range(N+1)]
        FV[:,M]=0
        FV[N,:]=np.maximum(K-np.arange(S_min,S_max+dS/2.0,dS,dtype=np.float),0)
    
    FV=np.matrix(np.array(FV))
    #Start the Backward Iteration
    for i in range(N-1,-1,-1):
        k_i=np.zeros((M,1))
        #Inserts the first and the last element
        k_i[0]=a[0]*FV[i+1,0] 
        k_i[M-1]=c[M-1]*FV[i+1,M]
        
        FV[i,0:M]=np.dot(FV[i+1,0:M],A)+np.matrix(np.array(k_i)).transpose()
        
        if(bool(isAmerican)):
            if(bool(isCall)):
                FV[i,:] = np.maximum(np.arange(S_min,S_max+dS/2,dS,dtype=float)-K,FV[i,:]) 
            else:
                FV[i,:] = np.maximum(K-np.arange(S_min,S_max+dS/2,dS,dtype=float),FV[i,:])
        outVector[0] = FV[0,(FV.shape[1]+1)/2]
    return outVector